In [1]:
import pandas as pd

In [2]:
df_sub = pd.read_csv('data/SampleSubmissionStage2.csv')
df_kp = pd.read_csv('data/Final/final_raw_kp_dat.csv')
df_seed = pd.read_csv('data/Final/final_raw_seeds.csv')

### Prepare Submission file 

To be a test dataset

In [3]:
df_sub[['Season','low_seed','high_seed']] = df_sub['ID'].str.split('_',expand=True).astype(int)

In [4]:
df_sub = pd.merge(df_sub,df_seed,how='inner',
                 left_on=['Season','low_seed'],
                 right_on=['Season','TeamID']).rename(columns={'Seed':'L_SEED'})
df_sub = pd.merge(df_sub,df_seed,how='inner',
                 left_on=['Season','high_seed'],
                 right_on=['Season','TeamID']).rename(columns={'Seed':'W_SEED'})

In [6]:
df_sub2 = df_sub.drop(['TeamID_x','TeamID_y'],axis=1)

In [7]:
df_test = df_sub2[['W_SEED','L_SEED']]

In [8]:
df_test['Seed_Diff'] = df_test['W_SEED'] - df_test['L_SEED']

/Users/meyappansubbaiah/Documents/Github/March_Madness_KG/.env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
df_test_final = df_test[['Seed_Diff']]

## Build Model

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
import numpy as np

In [12]:
df_train = pd.read_csv('data/Final/data_kp_seed.csv')

In [14]:
y = df_train['result'].values

In [15]:
X = df_train.drop('result',axis=1)

In [16]:
logreg = LogisticRegression(solver='lbfgs')
params = {'C': np.logspace(start=-5, stop=3, num=9)}
clf = GridSearchCV(logreg, params, scoring='neg_log_loss', refit=True)
clf.fit(X, y)
print('Best log_loss: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))

/Users/meyappansubbaiah/Documents/Github/March_Madness_KG/.env/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Best log_loss: -0.5502, with best C: 10.0


In [17]:
final_reg = LogisticRegression(**clf.best_params_)

In [18]:
final_reg.fit(X,y)

/Users/meyappansubbaiah/Documents/Github/March_Madness_KG/.env/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [19]:
df_final_sub = pd.read_csv('data/SampleSubmissionStage2.csv')

In [20]:
clipped_preds = np.clip(final_reg.predict_proba(df_test_final)[:,1], 0.05, 0.95)

In [21]:
df_final_sub['Pred'] = clipped_preds

In [26]:
%matplotlib inline

In [29]:
df_final_sub.to_csv('data/Final/Results/logreg_seed_diff.csv',index=False)

In [30]:
df_final_sub

,ID,Pred
0,2019_1101_1113,0.663260
1,2019_1101_1120,0.844834
2,2019_1101_1124,0.734346
3,2019_1101_1125,0.734346
4,2019_1101_1133,0.583930
5,2019_1101_1138,0.336740
6,2019_1101_1153,0.663260
7,2019_1101_1159,0.500000
8,2019_1101_1181,0.265654
9,2019_1101_1192,0.416070
